In [10]:
import numpy as np
import pandas as pd
import os

def error_metrics(file1, file2):
    """Return (MSE, MAE, SDE) for file1 vs file2, col 3."""
    stg1 = np.loadtxt(file1, delimiter=',', usecols=2)
    stg2 = np.loadtxt(file2, delimiter=',', usecols=2)
    err = stg1 - stg2
    mse = np.mean(err)
    mae = np.mean(np.abs(err))
    sde = np.std(err)
    return mse, mae, sde

# Reference file
ref_file = "../csv_files/TBE.csv"
y_ref = pd.read_csv(ref_file, header=None).iloc[:, 2].values

# Methods: original path, method name
methods = [
    ("../csv_files/AVDZ/B2GP-PLYP.csv", "B2GP-PLYP"),
    ("../csv_files/AVDZ/PBE-QIDH.csv", "PBE-QIDH"),
    ("../csv_files/AVDZ/L-ADC2.csv", "L-ADC(2)"),
    ("../csv_files/AVDZ/L-CC2.csv", "L-CC2")
]

scaled_paths = {}  # to map method name -> scaled file

# Scale and store results
coeffs = {}
for orig_path, method in methods:
    x_orig = pd.read_csv(orig_path, header=None).iloc[:, 2].values
    a, b = np.polyfit(x_orig, y_ref, 1)
    coeffs[method] = (a, b)

    scaled_x = np.round(a * x_orig + b, 3)
    scaled_data = pd.read_csv(orig_path, header=None)
    scaled_data.iloc[:, 2] = scaled_x
    scaled_name = f"{method.replace(' ', '').replace('(', '').replace(')', '')}_scaled.csv"
    scaled_data.to_csv(scaled_name, index=False, header=False)
    scaled_paths[method] = scaled_name


print("Scaling Coefficients:")
for method, (a, b) in coeffs.items():
    print(f"{method}: a = {a:.4f}, b = {b:.4f}")


row_mse = ["MSE"]
row_mae = ["MAE"]
row_sde = ["SDE"]

for orig_path, method in methods:
    mse_o, mae_o, sde_o = error_metrics(orig_path, ref_file)
    mse_s, mae_s, sde_s = error_metrics(scaled_paths[method], ref_file)
    row_mse.extend([f"{mse_o:.3f}", f"{mse_s:.3f}", ""])
    row_mae.extend([f"{mae_o:.3f}", f"{mae_s:.3f}", ""])
    row_sde.extend([f"{sde_o:.3f}", f"{sde_s:.3f}", ""])

header = ["Metric"]
for _, method in methods:
    header.extend([f"{method}", f"{method} (scaled)", ""])
print(" & ".join(header) + " \\\\")

print(" & ".join(row_mse) + " \\\\")
print(" & ".join(row_mae) + " \\\\")
print(" & ".join(row_sde) + " \\\\")


Scaling Coefficients:
B2GP-PLYP: a = 1.0654, b = -0.1190
PBE-QIDH: a = 1.0284, b = -0.1787
L-ADC(2): a = 0.9404, b = 0.0103
L-CC2: a = 0.9546, b = 0.0065
Metric & B2GP-PLYP & B2GP-PLYP (scaled) &  & PBE-QIDH & PBE-QIDH (scaled) &  & L-ADC(2) & L-ADC(2) (scaled) &  & L-CC2 & L-CC2 (scaled) &  \\
MSE & 0.119 & 0.000 &  & 0.177 & 0.000 &  & -0.019 & -0.000 &  & -0.013 & -0.000 &  \\
MAE & 0.119 & 0.007 &  & 0.177 & 0.008 &  & 0.023 & 0.014 &  & 0.016 & 0.010 &  \\
SDE & 0.011 & 0.009 &  & 0.009 & 0.009 &  & 0.017 & 0.016 &  & 0.013 & 0.012 &  \\
